# Detect Vessel Collisions
Now that we have our pipeline for taking a video of a microscope view of the brain during surgery and performing crops, color thresholding to produce masks, and recombining those back into gifs, its time to begin constructing the simulation in which our learner will live!

The first and most critical component of our simulation is to detect when the agent has selected a "lacing" site that collides with blood vasculature. This is my testing ground to figure out how to implement that

In [2]:
# ROS Kinetic Interferes with cv2 import

import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

In [14]:
import cv2
import argparse
import numpy as np
from numpy import savetxt
from fastai.vision import *
from PIL import Image
import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline

import os 
import glob
import imageio

In [4]:
# Useful function that was constructed the preserve the order of images since each image is related by the time
# dimension. 
def order_files_by_date(path_to_folder, file_type):
    files = glob.glob("%s*%s" % (path_to_folder, file_type))
    files.sort(key=os.path.getmtime)
    return files

In [6]:
lef

['/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/top_crop/_top_crop0.jpg',
 '/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/top_crop/_top_crop1.jpg',
 '/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/top_crop/_top_crop2.jpg',
 '/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/top_crop/_top_crop3.jpg',
 '/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/top_crop/_top_crop4.jpg',
 '/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/top_crop/_top_crop5.jpg',
 '/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/top_crop/_top_crop6.jpg',
 '/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/top_crop/_top_crop7.jpg',
 '/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/top_crop/_top_crop8.jpg',
 '/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/top_crop/_top_crop9.jpg',
 '/home/andrew/Githu

# Load Test Image

In [51]:
def get_example(image_files, index):
    return image_files[index]

In [53]:
image = get_example(top_segs, 0)

In [54]:
print(image.shape)

AttributeError: 'str' object has no attribute 'shape'

In [48]:
for i in range(len(image)):
    print(np.unique(image[i]))

[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255]
[255